<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/NEW_teststrat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install pandas matplotlib yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   380  100   380    0     0   1818      0 --:--:-- --:--:-- --:--:--  1818
100   359  100   359    0     0    913      0 --:--:-- --:--:-- --:--:--   913
  0   419    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   355  100   355    0     0    639      0 --:--:-- --:--:-- --:--:--   639
100 1299k  100 1299k    0     0  1095k      0  0:00:01  0:00:01 --:--:-- 2603k
ta-lib/
ta-lib/config.sub
ta-lib/aclocal.m4
ta-lib/CHANGELOG.TXT
ta-lib/include/
ta-lib/include/ta_abstract.h
ta-lib/include/ta_func.h
ta-lib/include/ta_common.h
ta-lib/include/ta_config.h.in
ta-lib/include/Makefile.am
ta-lib/include/ta_libc.h
ta-lib/include/ta_defs.h
ta-lib/missing
ta-lib/ta-lib.spec.in
ta-lib/config.guess
ta-lib/Makefile.in
ta-lib/ta-lib.dpkg.in
ta-lib/Makefile.am
ta-lib/autogen.sh
ta-lib/install-sh
ta-lib/configure
ta-lib/d

In [24]:
import pandas as pd
import talib as ta
import yfinance as yf
from datetime import date as date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt

In [25]:
file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

# ath_list

In [26]:
#!rm -rf database

In [ ]:
startdate = date.today() - relativedelta(months=1)
enddate = date.today() + relativedelta(days=1)

for company in ath_list:
  try:
    df = yf.download(company, start= startdate, end= enddate)
    df.to_csv(f"database/{company.split('.NS')[0]}.csv")
  except:
    pass


In [42]:
import os
datafiles = os.listdir('database')

for filename in datafiles:
  df = pd.read_csv(f"database/{filename}")
  try:
    result = ta.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'] )
    last = result.tail(1).values[0]
    if last > 0:
      print(f"{filename.split('.csv')[0]} was triggered , Bullish")
    elif last < 0:
      print(f"{filename.split('.csv')[0]} was triggered , Bearish")
  except:
    pass

#df_1d = df_1d.swaplevel(0, 1, 1).sort_index(axis=1)

DELHIVERY was triggered , Bearish
MCDOWELL-N was triggered , Bearish
NXTDIGITAL was triggered , Bearish
INDSWFTLTD was triggered , Bearish
JISLJALEQS was triggered , Bearish
VINATIORGA was triggered , Bearish
BVCL was triggered , Bearish
MORARJEE was triggered , Bearish
JMA was triggered , Bearish
ATUL was triggered , Bearish
ASHIANA was triggered , Bearish
MAHAPEXLTD was triggered , Bearish
PEL was triggered , Bearish
UNIONBANK was triggered , Bearish
BIGBLOC was triggered , Bullish
SUVENPHAR was triggered , Bullish
UTTAMSUGAR was triggered , Bearish
KIRLFER was triggered , Bearish
ROLEXRINGS was triggered , Bearish
CAMPUS was triggered , Bearish
SHREEPUSHK was triggered , Bearish
DIL was triggered , Bearish
PFIZER was triggered , Bearish
CONFIPET was triggered , Bearish
BAYERCROP was triggered , Bearish
VINYLINDIA was triggered , Bearish
NAGREEKCAP was triggered , Bearish
NELCO was triggered , Bearish
BODALCHEM was triggered , Bearish
JKPAPER was triggered , Bullish
FINEORG was trigg

In [29]:
# result = ta.CDLENGULFING(df_1d[:,'Open'], df_1d[:,'High'], df_1d[:,'Low'], df_1d[:,'Close'] )